# Multi-Agent Network Architecture for Complex Tasks

## Introduction

In this tutorial, we explore a powerful approach to handling complex tasks by leveraging a **multi-agent network architecture**. The core idea is to use a "divide-and-conquer" strategy, where specialized agents are created for specific tasks or domains, and tasks are routed to the appropriate "expert" agent. This approach allows for efficient problem-solving by breaking down complex tasks into smaller, manageable subtasks, each handled by an agent with the right expertise.

This tutorial demonstrates how to implement such a system using **LangGraph**, a framework for building multi-agent workflows. We will define specialized agents, such as a research agent for retrieving data and a chart generator agent for creating visualizations, and connect them in a collaborative network. By the end of this tutorial, you will understand how to design and deploy a multi-agent system to tackle complex, real-world problems effectively.

## Installation and Setup

Before we begin, we need to install the necessary libraries. These include LangChain, LangGraph, and other dependencies required for the agents to function.

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain-anthropic
!pip install -qU langchain_community
!pip install -qU langchain_experimental
!pip install -qU langgraph
!pip install -qU duckduckgo_search

## Defining Tools and Agents

In this section, we define the tools and agents that will be used in the multi-agent network. The tools include a DuckDuckGo search tool and a Python REPL tool for executing code. We also define the system prompts for the agents.

The code begins by importing necessary libraries, including `DuckDuckGoSearchRun` for web searches and `PythonREPL` for executing Python code. Two tools are then defined:

1. **DuckDuckGo Search Tool**:
   - This tool performs web searches using DuckDuckGo.
   - It takes a `query` and an optional `max_results` parameter (defaulting to 5) and returns the search results as a string.

2. **Python REPL Tool**:
   - This tool executes Python code locally using a Python REPL.
   - It takes a `code` string as input, executes it, and returns the result or an error message if execution fails.
   - A warning is included to highlight that executing code locally can be unsafe if not sandboxed properly.

Additionally, the `make_system_prompt` function generates a system prompt for the AI assistants. This prompt provides context for collaboration, instructs the assistants to use the provided tools, and specifies that they should prefix their response with `"FINAL ANSWER"` when the task is complete. The function appends additional instructions (`suffix`) to the base prompt for customization.

These tools and prompts form the foundation for the multi-agent network, enabling agents to retrieve data and execute code while collaborating effectively.

In [ ]:
from typing import Annotated
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

# Define DuckDuckGo Search Tool
@tool
def duckduckgo_search(query: str, max_results: int = 5) -> str:
    """Perform a search using DuckDuckGo and return the results.
    
    Args:
        query (str): The search query to be executed.
        max_results (int, optional): The maximum number of results to return. Defaults to 5.
    
    Returns:
        str: The search results as a string.
    """
    search = DuckDuckGoSearchRun()  # Initialize DuckDuckGoSearchRun
    results = search.run(query)     # Use run method for search
    return str(results)

# Warning: This executes code locally, which can be unsafe when not sandboxed
repl = PythonREPL()

@tool
def python_repl_tool(code: Annotated[str, "The python code to execute to generate your chart."]):
    """Execute Python code using a Python REPL (Read-Eval-Print Loop).
    
    Args:
        code (str): The Python code to execute.
    
    Returns:
        str: The result of the executed code or an error message if execution fails.
    """
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

# Create graph and define agent nodes
def make_system_prompt(suffix: str) -> str:
    """Generate a system prompt for the AI assistant.
    
    Args:
        suffix (str): Additional context or instructions to append to the base system prompt.
    
    Returns:
        str: The complete system prompt.
    """
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

## Creating the Multi-Agent Network

In this section, we create the multi-agent network using LangGraph. We define the research agent and the chart generator agent, and then we create the graph that connects these agents.

The code begins by importing necessary libraries, including `langchain_core.messages` for message handling, `langchain_openai` and `langchain_anthropic` for language models, and `langgraph` for building the multi-agent graph. The Anthropic API key is loaded to authenticate the language model (`claude-3-5-sonnet-latest`), which powers the agents.

Next, we define the logic for transitioning between nodes using the `get_next_node` function. This function checks if the last message contains `"FINAL ANSWER"` to determine whether to terminate the graph or proceed to the next node.

The **research agent** is then created, specializing in retrieving data using the DuckDuckGo search tool. It is given a specific task (`research_task`) and a system prompt to focus solely on research. The `research_node` function executes this agent, processes the search results, and transitions to the `chart_node` for further processing.

The **chart generator agent** is responsible for creating visualizations using the Python REPL tool. It follows detailed instructions (`chart_task`) to generate clear, visually appealing charts using libraries like `seaborn` and `plotly`. The `chart_node` function executes this agent, processes the chart generation, and transitions back to the `research_node` if additional research is needed.

Finally, the graph is constructed using `StateGraph` with `MessagesState` to manage the conversation state. Two nodes are added: `research_node` for data retrieval and `chart_node` for chart generation. The graph starts at the `research_node` and transitions to the `chart_node` after research is completed. The graph is then compiled into an executable workflow. Optionally, the graph can be visualized using Mermaid.js, though this step requires additional dependencies.

This modular approach enables efficient handling of complex tasks by leveraging specialized agents that collaborate within a structured workflow.

In [ ]:
from typing import Literal
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from kaggle_secrets import UserSecretsClient

# Load Anthropic API Key
my_api_key = UserSecretsClient().get_secret("my-anthropic-api-key")

#llm = ChatOpenAI(model="gpt-o1-mini", api_key=my_api_key)
llm = ChatAnthropic(model="claude-3-5-sonnet-latest", api_key=my_api_key)

def get_next_node(last_message: BaseMessage, goto: str):
    """Determine the next node to transition to based on the last message.
    
    Args:
        last_message (BaseMessage): The last message in the conversation.
        goto (str): The default node to transition to if no final answer is found.
    
    Returns:
        str: The next node to transition to, or END if a final answer is found.
    """
    if "FINAL ANSWER" in last_message.content:
        return END
    return goto

# Research agent and node
research_task = "You can only do research. You are working with a chart generator colleague."
research_agent = create_react_agent(llm, tools=[duckduckgo_search], state_modifier=make_system_prompt(research_task))

def research_node(state: MessagesState) -> Command[Literal["chart_node", END]]:
    """Execute the research node, which performs research using the DuckDuckGo search tool.
    
    Args:
        state (MessagesState): The current state of the conversation.
    
    Returns:
        Command: A command object containing the updated state and the next node to transition to.
    """
    result = research_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "chart_node")
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="research_node"
    )
    return Command(update={"messages": result["messages"]}, goto=goto)

# Chart generator agent and node
chart_task = """Create clear and visually appealing charts using seaborn and plotly. Follow these rules:
1. Add a title, labeled axes (with units), and a legend if needed.
2. Use `sns.set_context("notebook")` for readable text and themes like `sns.set_theme()` or `sns.set_style("whitegrid")`.
3. Use accessible color palettes like `sns.color_palette("husl")`.
4. Choose appropriate plots: `sns.lineplot()`, `sns.barplot()`, or `sns.heatmap()`.
5. Annotate key points (e.g., "Peak in 2020") for clarity.
6. Ensure the chart's width and display resolution is no wider than 1000px.
7. Display with `plt.show()`.
Goal: Produce accurate, engaging, and easy-to-interpret charts."""
chart_agent = create_react_agent(llm, [python_repl_tool], state_modifier=make_system_prompt(chart_task))

def chart_node(state: MessagesState) -> Command[Literal["research_node", END]]:
    """Execute the chart node, which generates charts using the Python REPL tool.
    
    Args:
        state (MessagesState): The current state of the conversation.
    
    Returns:
        Command: A command object containing the updated state and the next node to transition to.
    """
    result = chart_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "research_node")
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="chart_node"
    )
    return Command(update={"messages": result["messages"]}, goto=goto)

# Define the graph
from langgraph.graph import StateGraph, START

workflow = StateGraph(MessagesState)
workflow.add_node("research_node", research_node)
workflow.add_node("chart_node", chart_node)

workflow.add_edge(START, "research_node")
graph = workflow.compile()

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## Pretty-Printing Event Messages for Debugging

The `print_pretty` function is designed to format and display messages from events in a readable and structured way, making it easier to debug or log the interactions within the multi-agent network. Here's how it works:

1. **Input**: The function takes an `event` dictionary as input, which contains messages generated by the agents (e.g., `research_node` or `chart_node`).
2. **Node Key Check**: It checks if the event contains messages from specific nodes (`research_node` or `chart_node`). If found, it proceeds to process the messages.
3. **Message Extraction**:
   - For each message in the event, it extracts the message type (e.g., `HumanMessage`, `AIMessage`) using `message.__class__.__name__`.
   - The `content` of the message is extracted and formatted. If the content is a list, it is processed as a list of items. If it is a string, it is wrapped in quotes for better readability.
4. **Additional Fields**:
   - The function also extracts and prints additional metadata such as `additional_kwargs`, `response_metadata`, and `message.id` to provide more context about the message.
5. **Formatted Output**:
   - Each message is printed in a structured format, showing its type, content, metadata, and ID.
   - The output is indented and formatted for clarity, making it easy to read and analyze.
6. **Separator**:
   - A separator line (`"-" * 120`) is printed after each node's messages to visually distinguish between different nodes.
7. **Fallback**:
   - If no messages are found in the event, the function prints "No messages found in the event."

In [ ]:
def print_pretty(event):
    """Pretty-print the event messages for debugging or logging purposes.
    
    Args:
        event (dict): The event containing messages from the research or chart node.
    """
    # Check if the event contains 'research_node' or 'chart_node'
    for node_key in ["research_node", "chart_node"]:
        if node_key in event:
            messages = event[node_key].get("messages", [])
            print(f"{node_key}: [")
            for message in messages:
                # Extract message type (HumanMessage, AIMessage, etc.)
                message_type = message.__class__.__name__

                # Extract message content
                content = message.content
                if isinstance(content, list):
                    content = [item for item in content]  # Handle list content (e.g., AIMessage with tool use)
                elif isinstance(content, str):
                    content = f'"{content}"'  # Wrap string content in quotes

                # Extract additional fields
                additional_kwargs = message.additional_kwargs
                response_metadata = message.response_metadata
                message_id = message.id

                # Print the message in the desired format
                print(f"    {message_type}(")
                print(f"        content={content},")
                print(f"        additional_kwargs={additional_kwargs},")
                print(f"        response_metadata={response_metadata},")
                print(f"        id='{message_id}'")
                print( "    ),")
            print("]")
            print("-" * 120)
            return

    print("No messages found in the event.")

## Example 1: USA Population Growth Over Time

In this example, we use the multi-agent network to retrieve USA population data for the past 50 years and generate a line chart with annotations for significant events like economic recessions.

In [ ]:
# Invoke the graph
events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content="First, get the USA's population data for the past 50 years. "
                "Then, create a line chart with annotations for significant events like economic recessions. "
                "Add a trendline using numpy.polyfit. "
                "Once you make the chart, finish."
            )
        ],
    },
    {"recursion_limit": 150},
)

# Print events using print_pretty
for event in events:
    print_pretty(event)

## Example 2: Global CO2 Emissions by Country

In this example, we retrieve CO2 emissions data for the top 10 emitting countries over the past 20 years and create an interactive stacked area chart using Plotly.

In [ ]:
# Invoke the graph
prompt_message = ("""First, retrieve CO2 emissions data for the top 10 emitting countries over the past 20 years. 
Then, create a stacked area chart using Seaborn. 
Highlight the country with the highest emissions each year by making its line thicker and adding annotations. 
Ensure the chart includes a title, labeled axes, and a legend. 
Use a custom color palette for better visualization. 
Once the chart is created, display it with `plt.show()` and ensure the width is no wider than 1000px.""")

# Invoke the graph
events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content=prompt_message
            )
        ],
    },
    {"recursion_limit": 150},
)

# Print events using print_pretty
for event in events:
    print_pretty(event)

## Example 3: Weather Data Visualization

In this example, we retrieve temperature and precipitation data for New York City over the past 5 years and create a dual-axis chart with shaded regions for extreme weather events.

In [ ]:
# Invoke the graph
events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content="First, get temperature and precipitation data for New York City over the past 5 years. "
                "Then, create a dual-axis chart with temperature on the left y-axis and precipitation on the right y-axis. "
                "Add shaded regions for extreme weather events like heatwaves and heavy rainfall. "
                "Once you make the chart, finish."
            )
        ],
    },
    {"recursion_limit": 150},
)

# Print events using print_pretty
for event in events:
    print_pretty(event)

## Conclusion

This notebook demonstrates how to implement a `Multi-Agent Network Architecture` using `LangGraph`. By dividing tasks among specialized agents, we can effectively handle complex tasks that require multiple domains of expertise. This approach is highly flexible and can be adapted to various use cases, from data retrieval to visualization.